# Dealing With Open Ended Data

In [ ]:
!ls

Let's use the commandline `head` function to determine what the column headers for our data are. This will allow us to fetch the column number of the column we are interested in investigating. The value defined for the `-n` argument is the number of rows that we want to print out. In this case, it is only one row so we pass an argument of 1.

In [ ]:
!head -n 1 data/crimes.csv

We want to fetch information about the unique points in column 8. That's what the `-f` argument in the command below describes. The `-d` argument allows us to specify the parameter. The first part of our query, the part before the first `|`, allows us to extract the 8th row of data from the file. The second part of the command sorts this data, the third extracts out the unique values, and the last return the number of unique values that were found.

In [ ]:
!cut -d "," -f 8 data/crimes.csv | sort | uniq | wc -l

So it looks like we have 131 unique values in our Location Description column. What are those unique values? Well, to do that we just truncate the last part of our command out.

In [ ]:
!cut -d "," -f 8 data/crimes.csv | sort | uniq

Now let's see if we can get a count of each of the unique values in the Location Description.

In [ ]:
!awk -F ',' '{print $8}' data/crimes.csv | sort | uniq -c | sort -nr

Let's dissect this command a little bit. The `-F` alows us to dictate the delimiter that we are using and the `{print $8}` allows us to dictate that we want the awk command to print out the 8th column of the crimes file to the commands that is piped to. Again, sort operates on a lexical sort. What does the `-c` in the `uniq` command specify? It allows us to print out the number of times that each unique value occured. The last sort command allows us to sort the columns by the number of times they occur. In this particular case, the `n` allows us to sort on numerical values and the `r` sorts in reverse order.

It looks like we have some missing data in our dataset, similar to what we did with the latitude and longitude values, let's fill in these missing values with an artificial value. In this particular case, we'll categorize any missing values with the description "STREET" which corresponds with the most popular entry in this particular data set.

In [ ]:
!awk -F ',' '{if ($8 == "") print $8;}' data/crimes.csv | wc -l

Now in this particular case we are using the `awk` command again but notice that we filter out the data and print the value in the 8th column only if it is empty. We then pipe this output to `wc -l` which counts the number of lines in a file.

### Tagging By Freeform Content

Now that we have done some preliminary file cleaning at the command-line, let's pop back into Python and pandas to take care of the remaining tasks. Mainly, we'll be tagging each of the data points with a keyword that corresponds to a keyword found in the freeform entry.

In [2]:
import pandas as pd

In [3]:
crimes = pd.read_csv('data/crimes.csv')

Let's start off by filling in the missing location description values with the default value "STREET".

In [6]:
crimes['Location Description'] = crimes['Location Description'].fillna("STREET")

Now let's isolate the columns that relate to incidents that occurred on CTA property and tag them appropriately.

In [8]:
crimes.loc[crimes['Location Description'].str.contains('CTA'), "Location_Tag"] = "CTA"

Now you try! Create a tag that evaluates if the "Location Description" contains home-related terms such as "Residence" or "Apartment" and tag them appropriately.